In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from collections import Counter

In [2]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['label'] = iris.target
df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
# data = np.array(df.iloc[:100, [0, 1, -1]])

In [3]:
data = np.array(df.iloc[:100, [0, 1, -1]])
X, y = data[:,:-1], data[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [64]:
def knn_predict(k,x,y,target):
    knn_list=[]
    for i in range(k):
        dist=np.linalg.norm(target-x[i],ord=2)
        knn_list.append((dist,y[i]))
    for m in range(k,len(x)):
        max_index=knn_list.index(max(knn_list,key=lambda x:x[0]))
        dist2=dist=np.linalg.norm(target-x[m],ord=2)
        if knn_list[max_index][0]>dist2:
            knn_list[max_index]=(dist2,y[m])
    
    y_value=[n for m,n in knn_list]
    value_counter=Counter(y_value)
    max_value=sorted(value_counter)[-1]
    return max_value
        

In [65]:
test_point =[4.9 ,3.1]
knn_predict(3,X_train,y_train,test_point)

0.0

In [25]:
def predict_score(k,x_train,y_train,x_test,y_test):
    right_counter=0
    for i in range(len(y_test)):
        if y_test[i] == knn_predict(k,x_train,y_train,x_test[i]):
            right_counter+=1
    return right_counter/len(y_test)
    
        
    
    

In [66]:
predict_score(3,X_train,y_train,X_test,y_test)

1.0

In [89]:
from collections import namedtuple
from pprint import pformat

In [91]:
class kdd_nodes(namedtuple('nodes','split,location,left,right')):
    def __repr__(self):
        return pformat(tuple(self))
        

In [96]:
def createnode(data,split):
    if not data:
        return None
    
    
    k=len(data[0])
    data.sort(key=lambda x:x[0])
    m=len(data)//2
    median=data[m]
    split_next=(split+1) % k

    
    return kdd_nodes(split,
                     median,
                    createnode(data[:m],split_next),
                    createnode(data[m+1:],split_next))
    
    
    
    

In [98]:
data = [[2,3],[5,4],[9,6],[4,7],[8,1],[7,2]]
kd=createnode(data,0)
print(kd)

(0,
 [7, 2],
 (1, [4, 7], (0, [2, 3], None, None), (0, [5, 4], None, None)),
 (1, [9, 6], (0, [8, 1], None, None), None))


In [100]:
result=namedtuple('noderesult','point,dist,visited')

In [ ]:
def travel(tree,target,radius):
    k=len(target)
    if tree is None :
        return result([0]*k,float('inf'),0)
    
    page_visited=1
    page_split=tree.split
    page_point=tree.location
    
    if target[page_split]< page_point[page_split]:
        near_data=tree.left
        far_data=tree.right
    else:
        near_data=tree.right
        far_data=tree.left
    
    temp1=travel(near_data,target,radius)
    
    page_visited+=temp1.visited
    return_point=temp1.point
    return_dist=temp1.dist
    
    if return_dist<radius:
        radius=return_dist
        
    test_dist=np.abs(target[page_split]-page_point[page_split])
    
    if test_dist>radius:
        return result(return_point,return_dist,page_visited)
    
    